In [16]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# import pandas
import pandas as pd

In [2]:
# set the executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/joeymyers/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


### automate visit to Mars news site

In [4]:
# assign url and instruct the browser to visit it
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# search for elements w combo of div tag and list_text attribute
# tell browser to wait 1 sec before searching for components just in case it takes a bit to load pages
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# set up html parser
# html object
html = browser.html
news_soup = soup(html, 'html.parser')
# parent element - slide_elem holds a large chunk of info for first article: date, title, teaser
slide_elem = news_soup.select_one('div.list_text')

In [7]:
# search the parent element to find the first `a` tag and save it as `news_title`
# .get_text() returns us only text and none of the extra html
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"8 Martian Postcards to Celebrate Curiosity's Landing Anniversary"

In [9]:
# Search the parent element to find the teaser text
news_summary = slide_elem.find('div', class_='article_teaser_body').get_text()
news_summary

'The NASA rover touched down eight years ago, on Aug. 5, 2012, and will soon be joined by a second rover, Perseverance.'

### Featured Images

In [10]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [12]:
# Find and click the full image button - want to click 2nd button
full_image_elem = browser.find_by_tag('button')[1]
# splinter will click the image to view its fill size
full_image_elem.click()

In [13]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [14]:
# tell BS to look inside <img /> tag for an image w class of fancybox-image
# This is where the image we want lives- use .get('src') pulls the link to the image
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [15]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Mars Facts

In [21]:
# scrape entire table w pd function .read_html()
# func specifically looks for and returns tables found in html specify 1st table w [0]
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
# convert df back into html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [22]:
# end automated browser session
browser.quit()